## Camera Calibration

### Goal<br>
- Learn distortions in camera, intrinsic and extrinsic parameters of camera.
- Learn to find these parameters and undistort images.



Due to radial distortion, straight lines appears to be curved. This effect is more as we move away from the center of image.<br> 
For example, one image is shown below, where two edges of a chess board are marked with red lines. But you can see that border is not a straight line and doesn’t match with the red line. All the expected straight lines are bulged out.

![Radial Distortion](calib_radial.jpg "Radial Distortion")


**The distortion is solved as:**<br>
x<sub>{corrected}</sub> = x( 1 + k<sub>1</sub> r<sup>2</sup> + k<sub>2</sub> r<sup>4</sup> + k<sub>3</sub> r<sup>6</sup>) <br> 
y<sub>{corrected}</sub> = y( 1 + k<sub>1</sub> r<sup>2</sup> + k<sub>2</sub> r<sup>4</sup> + k<sub>3</sub> r<sup>6</sup>)
****

Similarly, another distortion is the **tangential distortion** which occurs because image taking lense is not aligned perfectly parallel to the imaging plane.<br> So some areas in image may look nearer than expected. <br><br>
**Tangential distortion is solved as:**<br>
x<sub>{corrected}</sub> = x + [ 2p<sub>1</sub>xy + p<sub>2</sub>(r<sup>2</sup> + 2x<sup>2</sup>)]
<br> 
y<sub>{corrected}</sub> = y + [p<sub>1</sub>(r<sup>2</sup>+ 2y<sup>2</sup>) +2 p<sub>2</sub>xy]

In short, we need to find five parameters, known as distortion coefficients given by:<br>

Distortion coefficients=(k<sub>1</sub>, k<sub>2</sub>, p<sub>1</sub>, p<sub>2</sub>, k<sub>3</sub>)<br>

In addition to this, we need to find a few more informatios like:<br> 
- Intrinsic parameters
- Extrinsic parameters of a camera.
<br>
Intrinsic parameters are specific to a camera. It includes information like focal length (f<sub>x</sub>,f<sub>y</sub>), optical centers (c<sub>x</sub>, c<sub>y</sub>) etc.<br> <br>
It is also called camera matrix and depends on the camera only, so once calculated, it can be stored for future purposes. It is expressed as a 3x3 matrix:

**camera matrix** = $$
\left(\begin{array}{cc} 
f_x & 0 & c_x\\
0 & f_y & c_y\\
0 & 0 & 1
\end{array}\right)
$$ 

Extrinsic parameters corresponds to rotation and translation vectors which translates a coordinates of a 3D point to a coordinate system.

For stereo applications, these distortions need to be corrected first. To find all these parameters, what we have to do is to provide some sample images of a well defined pattern (eg, chess board). <br>We find some specific points in it ( square corners in chess board). <br>We know its coordinates in real world space and we know its coordinates in image. For better results, we need atleast 10 test patterns.

> Consider just one image of a chess board. Important input datas needed for camera calibration is a set of 3D real world points and its corresponding 2D image points. 2D image points are OK which we can easily find from the image. (These image points are locations where two black squares touch each other in chess boards)

>What about the 3D points from real world space? Those images are taken from a static camera and chess boards are placed at different locations and orientations. So we need to know (X,Y,Z) values. But for simplicity, we can say chess board was kept stationary at XY plane, (so Z=0 always) and camera was moved accordingly. This consideration helps us to find only X,Y values. Now for X,Y values, we can simply pass the points as (0,0), (1,0), (2,0), ... which denotes the location of points. In this case, the results we get will be in the scale of size of chess board square. But if we know the square size, (say 30 mm), and we can pass the values as (0,0),(30,0),(60,0),..., we get the results in mm. (In this case, we don’t know square size since we didn’t take those images, so we pass in terms of square size).

3D points are called object points and 2D image points are called image points.

## 1 - Packages ##

First, let's run the cell below to import all the packages for camera calibration.
- [numpy] The fundamental package for scientific computing with Python.
- [cv2]   The computer vision 3 package in Python.
- [glob]


In [2]:
# This folder has sample chessboards images so we will use them
%matplotlib inline
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimage

We next write the termination conditions, define object as numpy array and load the images.

In [3]:
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('*.jpg')

In [5]:
# We use the images and then we convert them from 3 channel BGR to Gray.
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    # Next task is to find the chess board corners.
    ret, corners = cv2.findChessboardCorners(gray, (7,6),None)

    # If found then we add object points, image points after refining them
    if ret == True:
        objpoints.append(objp)

        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners2)
        
        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (7,6), corners2,ret)
        cv2.imshow('img',img)
        cv2.waitKey(500)
        plotimg=mpimg.imread(img)
        imgplot=plt.imshow(plotimg)
cv2.destroyAllWindows()



NameError: name 'mpimg' is not defined